# Space

In [1]:
import os
import sys 
import logging
import random
import pandas as pd 
from pprint import pprint 
from IPython.display import display, HTML

# WorkSpace
KEY = 'WorkSpace'; WORKSPACE_PATH = os.getcwd().split(KEY)[0] + KEY; print(WORKSPACE_PATH)
os.chdir(WORKSPACE_PATH)
sys.path.append(WORKSPACE_PATH)

# Pipeline Space
from proj_space import SPACE
SPACE['WORKSPACE_PATH'] = WORKSPACE_PATH
sys.path.append(SPACE['CODE_FN'])
pprint(SPACE)

# Available Packages
import argparse
import datasets
import pandas as pd
from datetime import datetime 

from recfldtkn.ckpd_obs import Ckpd_ObservationS
from recfldtkn.configfn import load_cohort_args
from recfldtkn.loadtools import load_module_variables, update_args_to_list
from recfldtkn.observer import get_RecObsName_to_RecObsInfo, CaseObserverTransformer

logger = logging.getLogger(__name__)
recfldtkn_config_path = os.path.join(SPACE['CODE_FN'], 'config_recfldtkn/')


/Users/floydluo/Library/CloudStorage/GoogleDrive-junjie.luo.jhu@gmail.com/Shared drives/CDHAI-WellDoc/2024-WellDocTest-SPACE/_WellDoc-AI-CGMGPT-v1-WorkSpace
{'CODE_FN': '../pipeline',
 'CODE_RFT': '../pipeline',
 'DATA_CaseFeat': '../_Data/3-Data_CaseFeat',
 'DATA_CaseObs': '../_Data/2-Data_CaseObs',
 'DATA_CaseSet': '../_Data/4-Data_CaseSet',
 'DATA_RAW': '../_Data/0-Data_Raw',
 'DATA_RFT': '../_Data/1-Data_RFT',
 'DATA_TASK': 'Data_CGMGPTv1',
 'MODEL_TASK': 'Model_CGMGPTv1',
 'WORKSPACE_PATH': '/Users/floydluo/Library/CloudStorage/GoogleDrive-junjie.luo.jhu@gmail.com/Shared '
                   'drives/CDHAI-WellDoc/2024-WellDocTest-SPACE/_WellDoc-AI-CGMGPT-v1-WorkSpace'}


# [Part 1] Get a Caseset: Case Examples

## [Step 1] Trigger Event

In [2]:
from recfldtkn.loadtools import fetch_TriggerEvent_tools

####################
TriggerCaseMethod = 'CGM5MinEntry'
####################

Trigger_tools = fetch_TriggerEvent_tools(TriggerCaseMethod, SPACE)
Trigger_tools 

{'TriggerRecName': 'CGM5Min',
 'case_id_columns': ['PID', 'ObsDT'],
 'special_columns': ['PID', 'DT_s'],
 'convert_TriggerEvent_to_Caseset': <function CGM5MinEntry.convert_TriggerEvent_to_Caseset(ds_rec, case_id_columns, special_columns, base_config)>}

In [3]:
##################################
CaseSetName = TriggerCaseMethod
case_id_columns = Trigger_tools['case_id_columns']
special_columns = Trigger_tools['special_columns']
##################################

TriggerCasePath = os.path.join(SPACE['DATA_CaseSet'], f'{CaseSetName}.p')
cohort_args = load_cohort_args(recfldtkn_config_path, SPACE)
cohort_args['Ckpd_ObservationS'] = Ckpd_ObservationS
print(cohort_args)
print(TriggerCasePath)

df_case = pd.read_pickle(TriggerCasePath) # (1000)
df_case = df_case.sample(100, random_state=0).reset_index(drop=True)
ds_case = datasets.Dataset.from_pandas(df_case)
ds_case

{'CohortInfo': {'RawData2022_CGM': {'cohort_label': 1, 'cohort_name': 'RawData2022_CGM', 'FolderPath': '../_Data/0-Data_Raw/RawData2022_CGM/'}, 'RawData2023_CVSTDCAug': {'cohort_label': 2, 'cohort_name': 'RawData2023_CVSTDCAug', 'FolderPath': '../_Data/0-Data_Raw/RawData2023_CVSTDCAug/'}, 'RawData2023_CVSDeRxAug': {'cohort_label': 3, 'cohort_name': 'RawData2023_CVSDeRxAug', 'FolderPath': '../_Data/0-Data_Raw/RawData2023_CVSDeRxAug/'}}, 'RawRootID': 'PatientID', 'RootID': 'PID', 'RecName': 'PRawRecNum', 'recattr_pyfolder': '../pipeline/fn_recattr/', 'fldtkn_pyfolder': '../pipeline/fn_fldtkn/', 'humanrec_pyfolder': '../pipeline/fn_humanrec/', 'inference_pyfolder': '../pipeline/fn_inference/', 'trigger_pyfolder': '../pipeline/fn_trigger/', 'RecName_to_RFT_GROUP_SIZE': {'CGM5Min': 100, 'Default': 5000}, 'RecName_to_RFT_idx_group_size': {'CGM5Min': 100, 'BGManual': 100, 'Default': 10000}, 'RecName_to_RFT_usebucket': {'CGM5Min': True, 'Default': False}, 'RootIDLength': 6, 'recfldtkn_config_p

Dataset({
    features: ['PID', 'ObsDT'],
    num_rows: 100
})

## [Step 2] Generate Arguments

In [4]:
#################################
case_observations = [
    'BfCGM:ro.CGM5Min-Bf24H-N2Cin1_ct.1TknIn5Mintid',
    'AfCGM:ro.CGM5Min-Af2H-N2Cin1_ct.1TknIn5Mintid',
]

name_CaseGamma = 'InputCatCOSeqType'
#################################

In [5]:
from pprint import pprint 
from recfldtkn.obsname import convert_case_observations_to_co_to_observation
from recfldtkn.obsname import get_RecNameList_and_FldTknList

co_to_COName, co_to_CONameInfo = convert_case_observations_to_co_to_observation(case_observations)
co_to_COName

{'BfCGM': 'ro.CGM5Min-Bf24H-N2Cin1_ct.1TknIn5Mintid',
 'AfCGM': 'ro.CGM5Min-Af2H-N2Cin1_ct.1TknIn5Mintid'}

In [6]:
# check whether this information is ready. 
PipelineInfo = get_RecNameList_and_FldTknList(co_to_CONameInfo)
pprint(PipelineInfo, sort_dicts=False)

{'RecNameList': ['CGM5Min'],
 'CkpdNameList': ['Af2H', 'Bf24H'],
 'FldTknList': ['CGM5Min-N2Cin1'],
 'CasePhiList': ['1TknIn5Mintid']}


In [7]:
COName_List = [CaseName for co, CaseName in co_to_COName.items()]

In [8]:
from recfldtkn.obsname import convert_CONameList_to_CFName

CaseFeatName = convert_CONameList_to_CFName(COName_List, name_CaseGamma)
print(CaseFeatName)

cf.InputCatCOSeqType_co.CqJnmX0x_d


In [9]:
# part of fetch_caseobs_Phi_tools
CF_Folder = os.path.join(SPACE['DATA_CaseFeat'], CaseFeatName)
if not os.path.exists(CF_Folder): os.makedirs(CF_Folder)
print(CF_Folder)

../_Data/3-Data_CaseFeat/cf.InputCatCOSeqType_co.CqJnmX0x_d


# [Part 2] Tools for ds_case_obs

## [Step 1] Prepare examples

In [10]:
case_examples = ds_case[:5]
print(case_examples)

{'PID': [1002840, 1002580, 1004670, 1003564, 1003011], 'ObsDT': [Timestamp('2021-12-07 13:50:00'), Timestamp('2021-11-10 12:15:00'), Timestamp('2021-11-13 06:40:00'), Timestamp('2021-09-15 13:35:00'), Timestamp('2021-10-17 17:15:00')]}


In [11]:
length = len(case_examples[list(case_examples.keys())[0]])
case_examples_list = [{k: v[i] for k, v in case_examples.items()} for i in range(length)]
idx_to_examples = {i: case_examples_list[i] for i in range(length)}
pprint(idx_to_examples, sort_dicts=False)

{0: {'PID': 1002840, 'ObsDT': Timestamp('2021-12-07 13:50:00')},
 1: {'PID': 1002580, 'ObsDT': Timestamp('2021-11-10 12:15:00')},
 2: {'PID': 1004670, 'ObsDT': Timestamp('2021-11-13 06:40:00')},
 3: {'PID': 1003564, 'ObsDT': Timestamp('2021-09-15 13:35:00')},
 4: {'PID': 1003011, 'ObsDT': Timestamp('2021-10-17 17:15:00')}}


In [12]:
case_example = idx_to_examples[0]
case_example

{'PID': 1002840, 'ObsDT': Timestamp('2021-12-07 13:50:00')}

## [Step 2] get_CF_id

In [13]:
# check whether this information is ready. 
PipelineInfo = get_RecNameList_and_FldTknList(co_to_CONameInfo)
pprint(PipelineInfo, sort_dicts=False)

{'RecNameList': ['CGM5Min'],
 'CkpdNameList': ['Af2H', 'Bf24H'],
 'FldTknList': ['CGM5Min-N2Cin1'],
 'CasePhiList': ['1TknIn5Mintid']}


In [14]:
import inspect
#################################################
def get_CF_id(case_example, CaseFeatName):
    PIDValue = case_example['PID']
    ObsDTValue = case_example['ObsDT'].isoformat()
    # RxIDValue = case_example['RxID']
    return f'{PIDValue}-{ObsDTValue}'
get_CF_id.fn_string = inspect.getsource(get_CF_id)
#################################################

## [Step 3] get_CaseObsName_to_CaseObsInfo

In [15]:
from recfldtkn.observer import get_CaseObsName_to_CaseObsInfo


record_to_ds_rec = {}        # set this to empty dictionary, then we will load data from disk
record_to_ds_rec_info = {}   # set this to empty dictionary, then we will load data from disk
COName_to_COInfo = get_CaseObsName_to_CaseObsInfo(COName_List,
                                                  SPACE, 
                                                  cohort_args, 
                                                  record_to_ds_rec, 
                                                  record_to_ds_rec_info)

In [16]:
for CaseObsName, CaseObsInfo in COName_to_COInfo.items():
    print(CaseObsName)
    print([i for i in CaseObsInfo])

ro.CGM5Min-Bf24H-N2Cin1_ct.1TknIn5Mintid
['RecObsName_List', 'name_CasePhi', 'get_selected_columns', 'CaseObsName', 'ROName_to_ROInfo', 'fn_CasePhi', 'get_CO_id', 'CO_Folder', 'CO_vocab']
ro.CGM5Min-Af2H-N2Cin1_ct.1TknIn5Mintid
['RecObsName_List', 'name_CasePhi', 'get_selected_columns', 'CaseObsName', 'ROName_to_ROInfo', 'fn_CasePhi', 'get_CO_id', 'CO_Folder', 'CO_vocab']


In [17]:
COName_to_co = {v: k for k, v in co_to_COName.items()}
COName_to_co

{'ro.CGM5Min-Bf24H-N2Cin1_ct.1TknIn5Mintid': 'BfCGM',
 'ro.CGM5Min-Af2H-N2Cin1_ct.1TknIn5Mintid': 'AfCGM'}

In [18]:
co_to_COvocab = {COName_to_co[COName]: CaseObsInfo['CO_vocab'] for COName, CaseObsInfo in COName_to_COInfo.items()}

for co, vocab in co_to_COvocab.items():
    print(co)
    print(vocab)

BfCGM
{'tid': {'tid2tkn': {0: 'unk', 1: 'BGValue:1~2', 2: 'BGValue:2~3', 3: 'BGValue:3~4', 4: 'BGValue:4~5', 5: 'BGValue:5~6', 6: 'BGValue:6~7', 7: 'BGValue:7~8', 8: 'BGValue:8~9', 9: 'BGValue:9~10', 10: 'BGValue:10~11', 11: 'BGValue:11~12', 12: 'BGValue:12~13', 13: 'BGValue:13~14', 14: 'BGValue:14~15', 15: 'BGValue:15~16', 16: 'BGValue:16~17', 17: 'BGValue:17~18', 18: 'BGValue:18~19', 19: 'BGValue:19~20', 20: 'BGValue:20~21', 21: 'BGValue:21~22', 22: 'BGValue:22~23', 23: 'BGValue:23~24', 24: 'BGValue:24~25', 25: 'BGValue:25~26', 26: 'BGValue:26~27', 27: 'BGValue:27~28', 28: 'BGValue:28~29', 29: 'BGValue:29~30', 30: 'BGValue:30~31', 31: 'BGValue:31~32', 32: 'BGValue:32~33', 33: 'BGValue:33~34', 34: 'BGValue:34~35', 35: 'BGValue:35~36', 36: 'BGValue:36~37', 37: 'BGValue:37~38', 38: 'BGValue:38~39', 39: 'BGValue:39~40', 40: 'BGValue:40~41', 41: 'BGValue:41~42', 42: 'BGValue:42~43', 43: 'BGValue:43~44', 44: 'BGValue:44~45', 45: 'BGValue:45~46', 46: 'BGValue:46~47', 47: 'BGValue:47~48', 48

## [Step 3] get_CF_vocab

In [19]:
##################################
def get_CF_vocab(co_to_COvocab):
    CF_vocab = {}
    co = [i for i in co_to_COvocab][0]
    CO_vocab = co_to_COvocab[co]
    CF_vocab = CO_vocab.copy()

    CF_vocab['input_ids'] = CF_vocab.pop('tid')
    return CF_vocab
##################################

get_CF_vocab.fn_string = inspect.getsource(get_CF_vocab)

In [20]:
CF_vocab = get_CF_vocab(co_to_COvocab)
for SeqType, vocab in CF_vocab.items():
    print(SeqType, ':', vocab)
    print('\n')

input_ids : {'tid2tkn': {0: 'unk', 1: 'BGValue:1~2', 2: 'BGValue:2~3', 3: 'BGValue:3~4', 4: 'BGValue:4~5', 5: 'BGValue:5~6', 6: 'BGValue:6~7', 7: 'BGValue:7~8', 8: 'BGValue:8~9', 9: 'BGValue:9~10', 10: 'BGValue:10~11', 11: 'BGValue:11~12', 12: 'BGValue:12~13', 13: 'BGValue:13~14', 14: 'BGValue:14~15', 15: 'BGValue:15~16', 16: 'BGValue:16~17', 17: 'BGValue:17~18', 18: 'BGValue:18~19', 19: 'BGValue:19~20', 20: 'BGValue:20~21', 21: 'BGValue:21~22', 22: 'BGValue:22~23', 23: 'BGValue:23~24', 24: 'BGValue:24~25', 25: 'BGValue:25~26', 26: 'BGValue:26~27', 27: 'BGValue:27~28', 28: 'BGValue:28~29', 29: 'BGValue:29~30', 30: 'BGValue:30~31', 31: 'BGValue:31~32', 32: 'BGValue:32~33', 33: 'BGValue:33~34', 34: 'BGValue:34~35', 35: 'BGValue:35~36', 36: 'BGValue:36~37', 37: 'BGValue:37~38', 38: 'BGValue:38~39', 39: 'BGValue:39~40', 40: 'BGValue:40~41', 41: 'BGValue:41~42', 42: 'BGValue:42~43', 43: 'BGValue:43~44', 44: 'BGValue:44~45', 45: 'BGValue:45~46', 46: 'BGValue:46~47', 47: 'BGValue:47~48', 48: 

## [Step 4] Fetch $o_{ij}$ List

In [21]:
pprint(idx_to_examples, sort_dicts=False)

{0: {'PID': 1002840, 'ObsDT': Timestamp('2021-12-07 13:50:00')},
 1: {'PID': 1002580, 'ObsDT': Timestamp('2021-11-10 12:15:00')},
 2: {'PID': 1004670, 'ObsDT': Timestamp('2021-11-13 06:40:00')},
 3: {'PID': 1003564, 'ObsDT': Timestamp('2021-09-15 13:35:00')},
 4: {'PID': 1003011, 'ObsDT': Timestamp('2021-10-17 17:15:00')}}


In [22]:
use_CO_from_disk = False
COName_to_FnCaseObsPhi = {}
for COName, COInfo in COName_to_COInfo.items():
    ro_to_ROName = None 
    FnCaseObsPhi = CaseObserverTransformer(ro_to_ROName,
                                            COInfo['ROName_to_ROInfo'], 
                                            COInfo['name_CasePhi'], 
                                            COInfo['fn_CasePhi'], 
                                            COInfo['CO_vocab'], 
                                            COInfo['get_CO_id'],
                                            COInfo['CO_Folder'], 
                                            df_case,
                                            use_CO_from_disk)
    COName_to_FnCaseObsPhi[COName] = FnCaseObsPhi

In [23]:
from recfldtkn.observer import CaseFeatureTransformer

fetch_examples_with_complete_COs = CaseFeatureTransformer.fetch_examples_with_complete_COs

use_CO_from_disk = False 
results = fetch_examples_with_complete_COs(idx_to_examples, 
                                            COName_to_co, 
                                            COName_to_COInfo, 
                                            COName_to_FnCaseObsPhi)

idx_to_examples, COName_to_FnCaseObsPhi = results

In [24]:
case_observations

['BfCGM:ro.CGM5Min-Bf24H-N2Cin1_ct.1TknIn5Mintid',
 'AfCGM:ro.CGM5Min-Af2H-N2Cin1_ct.1TknIn5Mintid']

In [25]:
for i, example in idx_to_examples.items():
    print(i)
    print(example)

0
{'PID': 1002840, 'ObsDT': Timestamp('2021-12-07 13:50:00'), 'BfCGM_tid': [221, 220, 221, 227, 234, 240, 246, 254, 261, 268, 274, 283, 290, 295, 298, 298, 296, 291, 286, 279, 269, 260, 259, 260, 263, 261, 258, 249, 241, 235, 225, 209, 186, 163, 143, 126, 115, 107, 101, 96, 92, 87, 78, 68, 63, 65, 74, 83, 89, 89, 85, 80, 75, 68, 59, 51, 47, 48, 52, 57, 61, 66, 71, 77, 82, 86, 92, 97, 103, 107, 114, 120, 126, 131, 135, 138, 140, 142, 142, 139, 134, 129, 126, 124, 123, 126, 129, 133, 138, 146, 155, 164, 170, 174, 177, 181, 185, 190, 195, 199, 200, 201, 201, 204, 208, 213, 218, 221, 220, 217, 213, 207, 202, 195, 192, 190, 192, 192, 189, 184, 177, 171, 164, 156, 147, 140, 137, 141, 146, 148, 147, 143, 138, 134, 129, 127, 125, 124, 123, 122, 120, 118, 117, 116, 114, 112, 110, 108, 107, 105, 103, 99, 96, 95, 95, 94, 92, 91, 91, 91, 89, 84, 77, 71, 67, 65, 64, 65, 67, 69, 72, 76, 89, 109, 130, 143, 146, 146, 145, 146, 149, 153, 156, 158, 158, 157, 156, 156, 157, 157, 154, 150, 147, 145, 144, 

In [26]:
[i for i in example]

['PID', 'ObsDT', 'BfCGM_tid', 'AfCGM_tid']

## [Step 3]* Develop $\Gamma$. (To Dev)

In [27]:
idx = 1
case_example = idx_to_examples[idx]
# print('case_example:', case_example)
pprint(case_example, sort_dicts=False)

{'PID': 1002580,
 'ObsDT': Timestamp('2021-11-10 12:15:00'),
 'BfCGM_tid': [245,
               253,
               262,
               267,
               267,
               271,
               278,
               288,
               290,
               290,
               289,
               292,
               293,
               287,
               280,
               269,
               263,
               256,
               248,
               237,
               227,
               219,
               210,
               194,
               184,
               180,
               179,
               171,
               168,
               169,
               171,
               164,
               157,
               146,
               136,
               126,
               121,
               117,
               112,
               107,
               109,
               118,
               136,
               156,
               176,
               200,
               231,

In [28]:
##################################
def fn_CaseGamma(case_example, co_to_COvocab, CF_vocab):
    # co_Y_list = [i for i in co_to_COvocab if 'Fut' in i or 'Af' in i]
    # co_X_list = [i for i in co_to_COvocab if i not in co_Y_list]
    
    result_case = {}

    # the order here is important
    # co: 'BfCGM', 'AfCGM'
    co_list = [i for i in co_to_COvocab]
    # 'input_ids' # 'food_ids' 'activity_ids' 'med_ids' 'lab_ids' 'note_ids' 'symptom_ids' 'vital_ids'
    SeqType_list = [i for i in CF_vocab]

    # get the input ids
    for SeqType in SeqType_list: 
        X_tid_total = []
        X_wgt_total = []
        tkn2tid_CF = CF_vocab[SeqType]['tkn2tid']

        if SeqType == 'input_ids':
            SeqType_co = 'tid'
        else:
            SeqType_co = SeqType
        for co in co_list:
            CO_vocab = co_to_COvocab[co]
            X_tid_co = case_example[co +'_tid']
            X_wgt_co = case_example.get(co +'_wgt', [])
            X_tkn_co = [CO_vocab[SeqType_co]['tid2tkn'][tid] for tid in X_tid_co]
            X_tid_CF = [tkn2tid_CF[i] for i in X_tkn_co]
            
            X_tid_total = X_tid_total + X_tid_CF
            X_wgt_total = X_wgt_total + X_wgt_co
            
        # version 1: for DL and ML
        result_case['input_ids']  = X_tid_total
        if len(X_wgt_total) > 0:
            result_case['input_wgts'] = X_wgt_total

    return result_case 
##################################

fn_CaseGamma.fn_string = inspect.getsource(fn_CaseGamma)

In [29]:
result = fn_CaseGamma(example, co_to_COvocab, CF_vocab)
for SeqType, SeqValue in result.items():
    print(SeqType, ':', len(SeqValue), SeqValue)
    print('\n')

input_ids : 313 [147, 140, 134, 128, 124, 120, 117, 113, 110, 109, 111, 117, 126, 136, 146, 158, 171, 183, 191, 196, 202, 208, 215, 219, 222, 221, 221, 219, 216, 215, 215, 217, 218, 217, 214, 209, 205, 200, 196, 191, 185, 180, 176, 172, 168, 163, 158, 154, 150, 146, 143, 139, 133, 128, 123, 120, 118, 116, 114, 113, 112, 112, 113, 113, 114, 115, 116, 116, 117, 117, 116, 116, 117, 118, 119, 120, 121, 124, 128, 134, 140, 146, 149, 149, 146, 142, 139, 138, 138, 139, 142, 144, 146, 147, 148, 148, 149, 150, 151, 151, 151, 148, 143, 139, 136, 134, 133, 131, 130, 129, 127, 126, 127, 129, 129, 129, 129, 134, 141, 149, 156, 160, 162, 163, 162, 161, 158, 154, 151, 150, 150, 150, 149, 148, 146, 143, 139, 136, 133, 131, 129, 126, 122, 117, 113, 110, 107, 104, 102, 99, 96, 94, 93, 93, 94, 99, 108, 120, 132, 143, 152, 159, 165, 170, 174, 176, 176, 174, 171, 170, 170, 171, 171, 170, 169, 166, 164, 162, 162, 161, 159, 157, 155, 152, 149, 146, 142, 138, 134, 132, 130, 129, 128, 127, 126, 125, 125, 124, 

## Save to Files

In [30]:
from recfldtkn.loadtools import convert_variables_to_pystirng, load_module_variables

prefix = ['import pandas as pd', 'import numpy as np']
fn_variables = [get_CF_id, get_CF_vocab, fn_CaseGamma]
pycode = convert_variables_to_pystirng(fn_variables = fn_variables, prefix = prefix)
pypath = os.path.join(SPACE['CODE_FN'], 'fn_casegamma', f'gamma_{name_CaseGamma}.py')
with open(pypath, 'w') as file: file.write(pycode)

# Create a HTML link and display it
full_path = os.path.join(WORKSPACE_PATH, pypath)
display(HTML(f'{pypath} <a href="{full_path}" target="_blank">Open File</a>'))

# Application

In [31]:
CaseFeatName

'cf.InputCatCOSeqType_co.CqJnmX0x_d'

In [32]:
from recfldtkn.observer import get_CaseFeatInfo_for_a_CaseFeatName

record_to_ds_rec = {}
record_to_ds_rec_info = {}

CaseFeatInfo = get_CaseFeatInfo_for_a_CaseFeatName(name_CaseGamma,
                                                   case_observations,
                                                   SPACE, 
                                                   cohort_args, 
                                                   record_to_ds_rec, 
                                                   record_to_ds_rec_info)
[i for i in CaseFeatInfo]

['CaseFeatName',
 'name_CaseGamma',
 'case_observations',
 'co_to_COName',
 'COName_to_COInfo',
 'COName_List',
 'PipelineInfo',
 'fn_CaseGamma',
 'get_CF_id',
 'get_CF_vocab',
 'CF_vocab',
 'CF_Folder']

In [33]:
import random 
from pprint import pprint
from datetime import datetime 
from recfldtkn.observer import CaseFeatureTransformer

In [34]:

co_to_COName = CaseFeatInfo['co_to_COName']
COName_to_COInfo = CaseFeatInfo['COName_to_COInfo']
name_CaseGamma = CaseFeatInfo['name_CaseGamma']
fn_CaseGamma = CaseFeatInfo['fn_CaseGamma']
CF_vocab = CaseFeatInfo['CF_vocab']
get_CF_id = CaseFeatInfo['get_CF_id']
CF_Folder = CaseFeatInfo['CF_Folder']
df_case = ds_case.to_pandas()
use_CF_from_disk = False
use_CO_from_disk = False
batch_size = CaseFeatInfo.get('batch_size', 1000)

FnCaseFeatGamma = CaseFeatureTransformer(co_to_COName,
                                          COName_to_COInfo, 
                                          name_CaseGamma, 
                                          fn_CaseGamma, 
                                          CF_vocab, 
                                          get_CF_id,
                                          CF_Folder, 
                                          df_case,
                                          use_CF_from_disk,
                                          use_CO_from_disk)


# Check Cache Functions

In [35]:
FnCaseFeatGamma.new_CFs

{}

In [40]:
start = datetime.now()
batch_size = 100
ds_casetkn = ds_case.map(FnCaseFeatGamma, 
                         batched = True, 
                         batch_size = batch_size, 
                         load_from_cache_file = False, 
                         new_fingerprint = CaseFeatName)
end = datetime.now()
print('Elipse Time: ', end - start)
print(ds_casetkn)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Elipse Time:  0:00:00.014678
Dataset({
    features: ['PID', 'ObsDT', 'input_ids'],
    num_rows: 100
})


In [39]:
start = datetime.now()
batch_size = 100
ds_casetkn = ds_case.map(FnCaseFeatGamma, 
                         batched = True, 
                         batch_size = batch_size, 
                         load_from_cache_file = False, 
                         new_fingerprint = CaseFeatName)
end = datetime.now()
print('Elipse Time: ', end - start)
print(ds_casetkn)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Elipse Time:  0:00:00.023319
Dataset({
    features: ['PID', 'ObsDT', 'input_ids'],
    num_rows: 100
})


In [41]:
print(CaseObsName)
random_int = random.randint(0, len(ds_casetkn))
print(random_int)
pprint(ds_casetkn[random_int])
print(CF_vocab)

ro.CGM5Min-Af2H-N2Cin1_ct.1TknIn5Mintid
65
{'ObsDT': Timestamp('2021-08-10 04:50:00'),
 'PID': 1001364,
 'input_ids': [121,
               121,
               120,
               118,
               116,
               114,
               112,
               111,
               109,
               107,
               104,
               99,
               96,
               94,
               93,
               90,
               85,
               82,
               82,
               84,
               86,
               86,
               86,
               86,
               87,
               88,
               90,
               90,
               90,
               89,
               90,
               91,
               92,
               92,
               92,
               92,
               93,
               93,
               93,
               93,
               89,
               85,
               83,
               85,
               87,
               88,
           

In [42]:
ds_casetkn[72]

{'PID': 1003635,
 'ObsDT': Timestamp('2021-10-14 09:40:00'),
 'input_ids': [127,
  128,
  127,
  125,
  124,
  123,
  122,
  120,
  117,
  115,
  113,
  110,
  106,
  102,
  99,
  96,
  94,
  92,
  90,
  88,
  85,
  83,
  82,
  81,
  79,
  77,
  76,
  75,
  74,
  72,
  70,
  69,
  68,
  69,
  70,
  71,
  71,
  70,
  70,
  72,
  76,
  81,
  85,
  88,
  92,
  95,
  98,
  99,
  101,
  103,
  104,
  106,
  109,
  112,
  115,
  117,
  118,
  119,
  119,
  120,
  120,
  121,
  122,
  124,
  126,
  127,
  124,
  121,
  118,
  116,
  114,
  112,
  110,
  107,
  106,
  104,
  101,
  98,
  95,
  92,
  89,
  89,
  89,
  90,
  90,
  89,
  85,
  82,
  80,
  79,
  78,
  79,
  81,
  83,
  86,
  88,
  90,
  92,
  93,
  94,
  94,
  94,
  93,
  93,
  93,
  95,
  95,
  95,
  96,
  98,
  100,
  103,
  105,
  111,
  116,
  119,
  116,
  112,
  110,
  111,
  112,
  112,
  111,
  110,
  107,
  102,
  98,
  95,
  93,
  92,
  92,
  92,
  93,
  94,
  98,
  99,
  100,
  99,
  98,
  92,
  84,
  78,
  77,
  79,
  

# Save

In [43]:
FnCaseFeatGamma.df_CF_info

,casefeat_idx_in_data
casefeat_id,


In [44]:
CF_Folder_data = FnCaseFeatGamma.CF_Folder_data
CF_Folder_data

'../_Data/3-Data_CaseFeat/cf.InputCatCOSeqType_co.CqJnmX0x_d/data'

In [45]:
FnCaseFeatGamma.save_new_CFs_to_disk(CF_Folder_data)

[INFO:2024-03-02 18:14:06,390:(observer.py@939 recfldtkn.observer)]: use_CF_from_disk is false and empty new_CFs


In [46]:
FnCaseFeatGamma.df_CF_info

,casefeat_idx_in_data
casefeat_id,


In [47]:
FnCaseFeatGamma.ds_CF_data

In [48]:
CFids = FnCaseFeatGamma.CFids
ds_CF_data, df_CF_info = FnCaseFeatGamma.load_CFs_from_disk(CF_Folder_data, CFids)

In [49]:
ds_CF_data

In [50]:
df_CF_info

,casefeat_idx_in_data
casefeat_id,


# Save Vocab

In [51]:
CF_vocab = FnCaseFeatGamma.CF_vocab
CF_vocab

{'input_ids': {'tid2tkn': {0: 'unk',
   1: 'BGValue:1~2',
   2: 'BGValue:2~3',
   3: 'BGValue:3~4',
   4: 'BGValue:4~5',
   5: 'BGValue:5~6',
   6: 'BGValue:6~7',
   7: 'BGValue:7~8',
   8: 'BGValue:8~9',
   9: 'BGValue:9~10',
   10: 'BGValue:10~11',
   11: 'BGValue:11~12',
   12: 'BGValue:12~13',
   13: 'BGValue:13~14',
   14: 'BGValue:14~15',
   15: 'BGValue:15~16',
   16: 'BGValue:16~17',
   17: 'BGValue:17~18',
   18: 'BGValue:18~19',
   19: 'BGValue:19~20',
   20: 'BGValue:20~21',
   21: 'BGValue:21~22',
   22: 'BGValue:22~23',
   23: 'BGValue:23~24',
   24: 'BGValue:24~25',
   25: 'BGValue:25~26',
   26: 'BGValue:26~27',
   27: 'BGValue:27~28',
   28: 'BGValue:28~29',
   29: 'BGValue:29~30',
   30: 'BGValue:30~31',
   31: 'BGValue:31~32',
   32: 'BGValue:32~33',
   33: 'BGValue:33~34',
   34: 'BGValue:34~35',
   35: 'BGValue:35~36',
   36: 'BGValue:36~37',
   37: 'BGValue:37~38',
   38: 'BGValue:38~39',
   39: 'BGValue:39~40',
   40: 'BGValue:40~41',
   41: 'BGValue:41~42',
   42

In [52]:
CF_Folder_vocab = FnCaseFeatGamma.CF_Folder_vocab
CF_Folder_vocab

'../_Data/3-Data_CaseFeat/cf.InputCatCOSeqType_co.CqJnmX0x_d/vocab.p'

In [53]:
df_Vocab = pd.DataFrame({CaseObsName: CF_vocab})
df_Vocab.to_pickle(CF_Folder_vocab)
df_Vocab

,ro.CGM5Min-Af2H-N2Cin1_ct.1TknIn5Mintid
input_ids,"{'tid2tkn': {0: 'unk', 1: 'BGValue:1~2', 2: 'B..."
